<a href="https://colab.research.google.com/github/WhiteHum/Application-security/blob/main/1_03_SQL_Crash_Course.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL Crash Course

## Overview

One of the most common data sources for our work will be SQL databases.  SQL, or Structured Query Language, has been around for decades.  Even though NoSQL databases have become popular, SQL is still the most efficient solution for many problems.

In this lab, you will learn how to interact with a SQL database using Python.  You will learn how to enumerate databases, tables, and determine the structure of datables.  Finally, you will have an opportunity to apply some of what we discussed regarding SQL Joins.

***This lab requires the virtual machine.  If it is not currently running, please get it started now!***

## Goals

By the end of this lab, you should be able to:

 * Connect to SQL databases
 * Determine the databases present
 * Determine the tructure of a database
 * Enumerate tables
 * Retrieve data
 * Execute SQL Joins
 
## Estimated Time: 30 - 45 minutes

Since our language of choice is Python, we will use Python to explore SQL databases as well.  Our library of choice is `psycopg2`, which is one of, if not the most, popular libraries for working with Postgres from Python.

# <img src="../images/task.png" width=20 height=20> Task 3.1

Let's begin by importing the library `psycopg2`.  Please use the following cell to do so:

In [ ]:
import psycopg2

# <img src="../images/task.png" width=20 height=20> Task 3.2

Now that the library is imported, we need to connect to a database.  This lab will make use of your virtual machine.  If you have not yet started it up, please do so now.  When the virtual machine finishes booting, you should be able to find the IP address for your virtual machine in the console window.  

![VM IP Address](images/vm-ip-address.png)

Please find that address and make note of it.  In fact, use the next cell to assign that address to the variable `SERVER_ADDRESS`.

In [ ]:
SERVER_ADDRESS = "192.168.100.129"  # This must be changed to the address of your VM!

To connect to the database, we must make use of the `psycopg2.connect()` method.  Calling `help(psycopg2.connect)` we find:

```
connect(dsn=None, connection_factory=None, cursor_factory=None, **kwargs)
    Create a new database connection.
    
    The connection parameters can be specified as a string:
    
        conn = psycopg2.connect("dbname=test user=postgres password=secret")
    
    or using a set of keyword arguments:
    
        conn = psycopg2.connect(database="test", user="postgres", password="secret")
    
    Or as a mix of both. The basic connection parameters are:
    
    - *dbname*: the database name
    - *database*: the database name (only as keyword argument)
    - *user*: user name used to authenticate
    - *password*: password used to authenticate
    - *host*: database host address (defaults to UNIX socket if not provided)
    - *port*: connection port number (defaults to 5432 if not provided)
    
    Using the *connection_factory* parameter a different class or connections
    factory can be specified. It should be a callable object taking a dsn
    argument.
    
    Using the *cursor_factory* parameter, a new default cursor factory will be
    used by cursor().
    
    Using *async*=True an asynchronous connection will be created. *async_* is
    a valid alias (for Python versions where ``async`` is a keyword).
    
    Any other keyword parameter will be passed to the underlying client
    library: the list of supported parameters depends on the library version.
```

Reading through the documentation we can see that we have two main alternatives.  We can either create a string with all of the parameters embedded, or we can call `connect()` with keyword arguments.  Let's use the keyword arguments which feels more pythonic.

# <img src="../images/task.png" width=20 height=20> Task 3.3

Using the following cell, establish a connection object named `db` that is connected to the Postgres server on your virtual machine.  The username and password you should use are `sans` and `training`.  Since we *must* specify a database, use the database `postgres` for now.

In [ ]:
db = psycopg2.connect(
    host = SERVER_ADDRESS,
    database = "postgres",
    user = "sans",
    password = "ssrc2022!"
)

***Note:*** While writing queries, if you end up generating a database error and after fixing your query it still appears not to function, *you may need to recreate your database connection to resolve the problem!*


Now that we are connected, let's find out which databases are available.

In a Postgres database, information about the databases that are on the server or in the cluster is stored in the table `pg_database` within the `postgres` database.  Since we're already connected to the `postgres` database, we just need to query the `pg_database` table.

To execute a query, in this case a `SELECT` statement, we need to take several steps using this driver:

 1. Create a *cursor*
 2. Execute a query using that cursor
 3. Retrieve the results from the cursor
 
What is a *cursor*?  You can think of it as a database analog to the same kind of cursor that you use when editing a text document.  It is a marker or a placeholder.  When we *create* a cursor, it is as though we have opened a document for editing, but our mouse has not yet clicked into the content.  When we click into the content, a cursor appears.  While this is not exactly the same, this illustration is close enough for our purposes.

Creating a cursor is pretty simple:

```
cursor = db.cursor()
```

That's it!  Now that we have one, we would need to do something with it.  This will usually involve the use of the `execute()` method on the cursor.  For example, if we wanted to query all of the columns from a table we could use something like:

```
cursor.execute("SELECT * from pg_database")
```

Notice that we have placed the database query inside of quotation marks.  Since SQL statements can become quite complex, we may choose to write them as follows:

```
cursor.execute("""
SELECT *
    FROM pg_database
""")
```

By using triple quotes, which is usually termed a *heredoc*, we can embed SQL that spans multiple lines with various indentures, allowing it to be fairly easy to follow.

Once we have executed the query, we can now retrieve the results.  While we can do this iteratively, for now let's use the `fetchall()` method for the cursor:

```
results = cursor.fetchall()
```


# <img src="../images/task.png" width=20 height=20> Task 3.4

Retrieve all of the rows from the `pg_database` table in the `postgres` database.  Print all of the results.

In [ ]:
cursor = db.cursor()
cursor.execute("""
SELECT * FROM pg_database
""")
results = cursor.fetchall()
_ = [print(i) for i in results]

(13425, 'postgres', 10, 6, 'C.UTF-8', 'C.UTF-8', False, True, -1, 13424, '479', '1', 1663, None)
(1, 'template1', 10, 6, 'C.UTF-8', 'C.UTF-8', True, True, -1, 13424, '479', '1', 1663, '{=c/postgres,postgres=CTc/postgres}')
(13424, 'template0', 10, 6, 'C.UTF-8', 'C.UTF-8', True, False, -1, 13424, '479', '1', 1663, '{=c/postgres,postgres=CTc/postgres}')
(16387, 'news', 16384, 6, 'C.UTF-8', 'C.UTF-8', False, True, -1, 13424, '479', '1', 1663, None)


If all has gone well, you should see four rows returned.  While we can make a guess as to what the values in the rows mean, there's a better way to find out.  If we were using the database console, this would be as trivial as executing the command `DESCRIBE TABLE pg_database`.  Unfortunately for us, we don't have command line access.  (Well, we do, but we are trying to approach this as you would when attempting to interrogate a database remotely when you *are not* an administrator but only have *read* access.)

The information about the database table structure is stored in the `information_schema.columns` table in Postgres.  There is a tremendous amount of data stored in this database and table, so we will want to extract only a few columns:

 * `table_name`
 * `column_name`
 * `data_type`


# <img src="images/task.png" width=20 height=20> Task 3.5
 
Create a cursor and execute a query to discover the structure of the `pg_database` table.  Print the results.


In [ ]:
cursor = db.cursor()
cursor.execute("""
SELECT table_name, column_name, data_type
    FROM information_schema.columns
    WHERE table_name = 'pg_database'
""")
results = cursor.fetchall()
_ = [print(i) for i in results]

('pg_database', 'oid', 'oid')
('pg_database', 'datname', 'name')
('pg_database', 'datdba', 'oid')
('pg_database', 'encoding', 'integer')
('pg_database', 'datcollate', 'name')
('pg_database', 'datctype', 'name')
('pg_database', 'datistemplate', 'boolean')
('pg_database', 'datallowconn', 'boolean')
('pg_database', 'datconnlimit', 'integer')
('pg_database', 'datlastsysoid', 'oid')
('pg_database', 'datfrozenxid', 'xid')
('pg_database', 'datminmxid', 'xid')
('pg_database', 'dattablespace', 'oid')
('pg_database', 'datacl', 'ARRAY')


Excellent!  The names used for the columns aren't really intuitive, but they definitely follow a pattern.  Many of them begin with the first three letters *dat*, likely abbreviating "data" or "database."  The column that looks as though it will have the database names is the `datname` column.


# <img src="../images/task.png" width=20 height=20> Task 3.6
 
Create a query to retrieve *only* the database names from `pg_database` and print them out.

In [ ]:
cursor = db.cursor()
cursor.execute("""
SELECT datname
    FROM pg_database
""")
results = cursor.fetchall()
_ = [print(i) for i in results]

('postgres',)
('template1',)
('template0',)
('news',)


The database of interest for us is the *news* database.  

Information about the tables in databases is stored in another special Postgres database, `pg_catalog`.  More specifically, we are interested in the `pg_tables` table within the `pg_catalog` database.  We can interrogate the `information_schema.columns` table to figure out which tables are available in a database.  To do so, we will need to use a `WHERE` clause using the appropriate column name that identifies the table of interest.

# <img src="../images/task.png" width=20 height=20> Task 3.7
 
Write code to determine the name of the field in `pg_catalog.pg_tables` that contains the name of the table that any field is a part of.

In [ ]:
cursor = db.cursor()
cursor.execute("""
SELECT column_name
    FROM information_schema.columns
    WHERE table_name = 'pg_tables'
""")
results = cursor.fetchall()
_ = [print(i) for i in results]

('schemaname',)
('tablename',)
('tableowner',)
('tablespace',)
('hasindexes',)
('hasrules',)
('hastriggers',)
('rowsecurity',)


Once you figure this last task out, you may begin to notice some inconsistencies.  For example, the `information_schema.columns` table uses the name `table_name` to refer to the table, while `pg_tables` uses the column name `tablename`.  As unpleasant as this has been so far, the ability to find things like these names is the reason that we are taking this approach in investigating SQL databases.

Let's turn our attention to the *news* database.  We'd like to enumerate the tables that are contained in this database.  This information is stored in the `information_schema` database.  More specifically, it is inside of the `information_schema.tables` table.

This table contains information on every table within the database server.  In our case, much of this is for internal databases and tables.  How can we easily identify tables that are *not* part of the database system itself?

Within this table there is a column named `table_schema`.  All of the values within this column that begin `pg_` are internal.  Tables that are a part of some other user or application facing database will have the value `public`.  Perfect!  This gives us a way that we can find tables that are a part of databases that we are interested in.

In our sample database, there is only one public table.  What if that isn't the case?  What if there are dozens or hundreds of databases?  How can we find only the tables connected to the database of interest?  This same table, `information_schema.tables`, has another column named `table_catalog`.  The `table_catalog` value is the name of the database that the table is a part of.

# <img src="../images/task.png" width=20 height=20> Task 3.8
 
Reconnect to the database server, this time using the `news` database.  Retrieve and print out the names of all of the tables within the *news* database.

In [ ]:
db = psycopg2.connect(
    host = SERVER_ADDRESS,
    database = "news",
    user = "sans",
    password = "ssrc2022!"
)
cursor = db.cursor()
cursor.execute("""
SELECT table_catalog, table_name 
    FROM information_schema.tables
    WHERE table_schema = 'public'
""")
results = cursor.fetchall()
_ = [print(i) for i in results]

('news', 'categories')
('news', 'headlines')
('news', 'summaries')


Now that we have the names of the tables in the `news` database, let's look at each table and see what the names of the fields are.

# <img src="../images/task.png" width=20 height=20> Task 3.9
 
Write a function that will return a list of the column names for a table from a database.  Retrieve and print the column names for each of the tables in the `news` database.

In [ ]:
def column_names(table):
    cursor = db.cursor()
    cursor.execute(f"""
    SELECT table_name, column_name, data_type
        FROM information_schema.columns
        WHERE table_name = '{table}'
    """)
    results = cursor.fetchall()
    return results

for table in ['categories', 'headlines', 'summaries']:
    print(f"Table: {table}")
    _ = [print(i) for i in column_names(table)]

Table: categories
('categories', 'category_id', 'integer')
('categories', 'category_name', 'character varying')
Table: headlines
('headlines', 'headline_id', 'integer')
('headlines', 'category_id', 'integer')
('headlines', 'summary_id', 'integer')
('headlines', 'date', 'date')
('headlines', 'headline', 'character varying')
('headlines', 'url', 'character varying')
Table: summaries
('summaries', 'summary_id', 'integer')
('summaries', 'summary', 'character varying')


Let's try to select some data out of the database.  Recall that the general form for a select is:

```
SELECT [columns | *] FROM table[,table2...] [WHERE <conditions>]
```

We can use this to load data out of our database.

# <img src="../images/task.png" width=20 height=20> Task 3.10
 
Create a database cursor and use it to execute a `SELECT` query, retrieving all of the category names from the *categories* table.  Print out the category names.

In [ ]:
db = psycopg2.connect(
    host = SERVER_ADDRESS,
    database = "news",
    user = "sans",
    password = "ssrc2022!"
)

cursor = db.cursor()
cursor.execute("""
SELECT category_name
    FROM categories
""")
results = cursor.fetchall()
_ = [print(i) for i in results]

('CRIME',)
('ENTERTAINMENT',)
('WORLD NEWS',)
('IMPACT',)
('POLITICS',)
('WEIRD NEWS',)
('BLACK VOICES',)
('WOMEN',)
('COMEDY',)
('QUEER VOICES',)
('SPORTS',)
('BUSINESS',)
('TRAVEL',)
('MEDIA',)
('TECH',)
('RELIGION',)
('SCIENCE',)
('LATINO VOICES',)
('EDUCATION',)
('COLLEGE',)
('PARENTS',)
('ARTS & CULTURE',)
('STYLE',)
('GREEN',)
('TASTE',)
('HEALTHY LIVING',)
('THE WORLDPOST',)
('GOOD NEWS',)
('WORLDPOST',)
('FIFTY',)
('ARTS',)
('WELLNESS',)
('PARENTING',)
('HOME & LIVING',)
('STYLE & BEAUTY',)
('DIVORCE',)
('WEDDINGS',)
('FOOD & DRINK',)
('MONEY',)
('ENVIRONMENT',)
('CULTURE & ARTS',)


Next, let's look at the *headlines* table.  From the work that we did above, we know that the fields in this table include the `category_id`, `summary_id`, `headline`, `url`, and `date`.

# <img src="../images/task.png" width=20 height=20> Task 3.11
 
Retrieve and print out the first ten rows of the *headlines* table.

***Hint:*** While we could retrieve the entire result set and then limit our output to only ten rows, we can also use the `LIMIT` option when executing our query.  This is one of the many optional values that we can send in a `SELECT` statement.  It will limit the result set returned to a maximum of the number specified.

In [ ]:
cursor = db.cursor()
cursor.execute("""
SELECT *
    FROM headlines LIMIT 10
""")
results = cursor.fetchall()
_ = [print(i) for i in results]

(1, 'There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV', 1, 1, 'https://www.huffingtonpost.com/entry/texas-amanda-painter-mass-shooting_us_5b081ab4e4b0802d69caad89', datetime.date(2018, 5, 26))
(2, "Will Smith Joins Diplo And Nicky Jam For The 2018 World Cup's Official Song", 2, 2, 'https://www.huffingtonpost.com/entry/will-smith-joins-diplo-and-nicky-jam-for-the-official-2018-world-cup-song_us_5b09726fe4b0fdb2aa541201', datetime.date(2018, 5, 26))
(3, 'Hugh Grant Marries For The First Time At Age 57', 2, 3, 'https://www.huffingtonpost.com/entry/hugh-grant-marries_us_5b09212ce4b0568a880b9a8c', datetime.date(2018, 5, 26))
(4, "Jim Carrey Blasts 'Castrato' Adam Schiff And Democrats In New Artwork", 2, 4, 'https://www.huffingtonpost.com/entry/jim-carrey-adam-schiff-democrats_us_5b0950e8e4b0fdb2aa53e675', datetime.date(2018, 5, 26))
(5, 'Julianna Margulies Uses Donald Trump Poop Bags To Pick Up After Her Dog', 2, 5, 'https://www.huffingtonpost.com/entry/julianna-margulies-tr

We are now successfully reading data out of database tables, but this isn't where the real power of SQL lies.  The real power is in creating and leveraging *relations*.

What we'd like to do is retrieve the first ten headlines with the associated categories.  There are two ways that we can accomplish.  The first is leveraging the `WHERE` clause in the SQL `SELECT` statement:

```
SELECT table_1.column_a, table_2.column_b 
    FROM table_1, table_2
    WHERE table_1.table_2_foreign_key=table_2.id_column
```

In other words, extract the column `column_a` from *table_1* and `column_b` from *table_2*.  The `WHERE` clause aligns or joins these two tables so that the column read from *table_2* will depend on the value of the foreign key field in *table_1*.

Alternatively, we could write this as:

```
SELECT column_a, column_b_from_table_2
    FROM table_1
    INNER JOIN table_2 ON (table_1.table_2_foreign_key=table_2.id_column)
```

These two alternatives are equivalent and, in many ways, it is simply a matter of preference.  However, we would suggest that when you are performing a `JOIN` you are likely better explicitly calling it a `JOIN` rather than using `WHERE`.

# <img src="../images/task.png" width=20 height=20> Task 3.12

Retrieve the first ten rows from the *headlines* table along with the matching `category_name` value from the *categories* table.  Print out the category name and the headline for each.

In [ ]:
cursor = db.cursor()
cursor.execute("""
SELECT category_name, headline
    FROM headlines
    INNER JOIN categories ON (headlines.category_id=categories.category_id)
    LIMIT 10
""")
results = cursor.fetchall()
_ = [print(i) for i in results]

('CRIME', 'There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV')
('ENTERTAINMENT', "Will Smith Joins Diplo And Nicky Jam For The 2018 World Cup's Official Song")
('ENTERTAINMENT', 'Hugh Grant Marries For The First Time At Age 57')
('ENTERTAINMENT', "Jim Carrey Blasts 'Castrato' Adam Schiff And Democrats In New Artwork")
('ENTERTAINMENT', 'Julianna Margulies Uses Donald Trump Poop Bags To Pick Up After Her Dog')
('ENTERTAINMENT', "Morgan Freeman 'Devastated' That Sexual Harassment Claims Could Undermine Legacy")
('ENTERTAINMENT', "Donald Trump Is Lovin' New McDonald's Jingle In 'Tonight Show' Bit")
('ENTERTAINMENT', 'What To Watch On Amazon Prime That’s New This Week')
('ENTERTAINMENT', "Mike Myers Reveals He'd 'Like To' Do A Fourth Austin Powers Film")
('ENTERTAINMENT', 'What To Watch On Hulu That’s New This Week')


Our final task is to add to our existing query, now including the `summary` from the *summaries* table.

# <img src="../images/task.png" width=20 height=20> Task 3.13

Update your query to return the headline, category name, and summary for the first ten headlines in the database.


In [ ]:
cursor = db.cursor()
cursor.execute("""
SELECT category_name, headline, summary
    FROM headlines
    INNER JOIN categories ON (headlines.category_id=categories.category_id)
    INNER JOIN summaries ON (headlines.summary_id=summaries.summary_id)
    LIMIT 10
""")
results = cursor.fetchall()
_ = [print(i) for i in results]

('CRIME', 'There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV', 'She left her husband. He killed their children. Just another day in America.')
('ENTERTAINMENT', "Will Smith Joins Diplo And Nicky Jam For The 2018 World Cup's Official Song", 'Of course it has a song.')
('ENTERTAINMENT', 'Hugh Grant Marries For The First Time At Age 57', 'The actor and his longtime girlfriend Anna Eberstein tied the knot in a civil ceremony.')
('ENTERTAINMENT', "Jim Carrey Blasts 'Castrato' Adam Schiff And Democrats In New Artwork", 'The actor gives Dems an ass-kicking for not fighting hard enough against Donald Trump.')
('ENTERTAINMENT', 'Julianna Margulies Uses Donald Trump Poop Bags To Pick Up After Her Dog', 'The "Dietland" actress said using the bags is a "really cathartic, therapeutic moment."')
('ENTERTAINMENT', "Morgan Freeman 'Devastated' That Sexual Harassment Claims Could Undermine Legacy", '"It is not right to equate horrific incidents of sexual assault with misplaced compliment